<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#라이브러리-로딩" data-toc-modified-id="라이브러리-로딩-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>라이브러리 로딩</a></span></li><li><span><a href="#전처리" data-toc-modified-id="전처리-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>전처리</a></span><ul class="toc-item"><li><span><a href="#전처리를-위한-정규표현식-패턴-정의" data-toc-modified-id="전처리를-위한-정규표현식-패턴-정의-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>전처리를 위한 정규표현식 패턴 정의</a></span></li><li><span><a href="#학습-데이터셋-로드" data-toc-modified-id="학습-데이터셋-로드-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>학습 데이터셋 로드</a></span></li><li><span><a href="#Pos-tagging" data-toc-modified-id="Pos-tagging-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Pos tagging</a></span></li><li><span><a href="#Doc2vec-입력객체-생성" data-toc-modified-id="Doc2vec-입력객체-생성-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Doc2vec 입력객체 생성</a></span></li></ul></li><li><span><a href="#모델-hyperparameter-설정-및-학습" data-toc-modified-id="모델-hyperparameter-설정-및-학습-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>모델 hyperparameter 설정 및 학습</a></span></li><li><span><a href="#학습결과-확인" data-toc-modified-id="학습결과-확인-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>학습결과 확인</a></span><ul class="toc-item"><li><span><a href="#단어-유사도-확인" data-toc-modified-id="단어-유사도-확인-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>단어 유사도 확인</a></span></li><li><span><a href="#테스트-데이터셋-로드-및-pos-tagging" data-toc-modified-id="테스트-데이터셋-로드-및-pos-tagging-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>테스트 데이터셋 로드 및 pos tagging</a></span></li><li><span><a href="#테스트-결과-확인" data-toc-modified-id="테스트-결과-확인-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>테스트 결과 확인</a></span></li></ul></li><li><span><a href="#발생할-수-있는-문제점-및-원인" data-toc-modified-id="발생할-수-있는-문제점-및-원인-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>발생할 수 있는 문제점 및 원인</a></span><ul class="toc-item"><li><span><a href="#새로운-유형의-단어로-인해-발생하는-오류" data-toc-modified-id="새로운-유형의-단어로-인해-발생하는-오류-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>새로운 유형의 단어로 인해 발생하는 오류</a></span></li><li><span><a href="#Inference-시행-때마다-일정하지-않은-유사도-결과값-(Non-deterministic)" data-toc-modified-id="Inference-시행-때마다-일정하지-않은-유사도-결과값-(Non-deterministic)-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Inference 시행 때마다 일정하지 않은 유사도 결과값 (Non-deterministic)</a></span></li></ul></li></ul></div>

# 라이브러리 로딩

In [1]:
import MeCab
import re
import pandas as pd
from gensim.models import doc2vec

# 전처리

## 전처리를 위한 정규표현식 패턴 정의

In [2]:
mecab = MeCab.Tagger('-d ./Mecab/')  # mecab 사전 정의
pos_ptn = re.compile(r'.+\t[NVM][A-Z]{1,2}')  # pos tagging 패턴 
remove_ptn = re.compile(r'[^a-zㄱ-힣]')  # 불용어 패턴

## 학습 데이터셋 로드

In [3]:
train_docs = pd.read_csv('./ratings_train.txt', sep='\t')
train_docs = train_docs.dropna()
train_docs.index = range(len(train_docs))
train_docs.tail()

,id,document,label
149990,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149991,8549745,평점이 너무 낮아서...,1
149992,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149993,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1
149994,9619869,한국 영화 최초로 수간하는 내용이 담긴 영화,0


In [4]:
# 학습자료의 긍정, 부정문서 수 확인
train_docs.groupby(['label']).size()

label
0    75170
1    74825
dtype: int64

## Pos tagging

In [5]:
def pos_tagging(doc):
    # doc = string(문서 1개 내용)
    doc = remove_ptn.sub(' ', doc)
    parse_doc = mecab.parse(doc)
    pos_doc = pos_ptn.finditer(parse_doc)
    return [x[0].replace('\t', '\\') for x in pos_doc]

In [6]:
pos_docs = [pos_tagging(doc) for doc in train_docs['document']]
train_docs['prep_docs'] = pos_docs
train_docs = train_docs.dropna()
train_docs.index = range(len(train_docs))
train_docs.tail()

,id,document,label,prep_docs
149990,6222902,인간이 문제지.. 소는 뭔죄인가..,0,"[인간\NNG, 문제지\NNG, 소\NNG, 뭔\MM, 죄\NNG, 인가\VCP]"
149991,8549745,평점이 너무 낮아서...,1,"[평점\NNG, 너무\MAG, 낮\VA]"
149992,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0,"[이게\NP, 요\MM, 한국인\NNG, 거들먹거리\VV, 필리핀\NNP, 혼혈\N..."
149993,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1,"[청춘\NNG, 영화\NNG, 최고봉\NNG, 방황\NNG, 우울\NNG, 날\NN..."
149994,9619869,한국 영화 최초로 수간하는 내용이 담긴 영화,0,"[한국\NNP, 영화\NNG, 최초\NNG, 수간\NNG, 내용\NNG, 담긴\VV..."


## Doc2vec 입력객체 생성
- do2vec.TaggedDocument(단어 리스트, 라벨 리스트)

In [7]:
def doc2vec_labeler(docs):
    # docs = 학습 데이터셋 pandas DataFrame
    labeled_docs = list()
    for i in range(len(docs)):
        doc = docs.iloc[i].tolist()
        words = doc[3]
        labels = [str(doc[0]), str(doc[2])]
        # Doc2vec tagged object 생성
        sentence = doc2vec.TaggedDocument(words=words, tags=labels)
        labeled_docs.append(sentence)
    return labeled_docs

In [8]:
train_labeled_docs = doc2vec_labeler(train_docs)

In [9]:
train_labeled_docs[0]

TaggedDocument(words=['더\\MAG', '빙\\MAG', '진짜\\MAG', '짜증\\NNG', '나\\NP', '네요\\VCP', '목소리\\NNG'], tags=['9976970', '0'])

# 모델 hyperparameter 설정 및 학습
- dm=0: DBOW 모델 사용
- dm_tag_count=2: 문서 당 라벨의 수(예제에는 문서번호, 긍부정 여부 2개)

In [28]:
model = doc2vec.Doc2Vec(window=5, size=100, alpha=0.1, min_alpha=0.01, dm=0, 
                        min_count=3, dm_tag_count=2, workers=8)
model.build_vocab(train_labeled_docs)

In [11]:
# 말뭉치의 단어 수
len(model.wv.vocab)

21447

In [29]:
model.train(train_labeled_docs, total_examples=model.corpus_count, epochs=model.iter)

7813611

# 학습결과 확인

## 단어 유사도 확인
- DBOW는 word order를 무시
- 문서 벡터는 상대적으로 품질이 좋은 반면, 단어 벡터는 직관과 어긋나는 결과를 주로 보임.

In [33]:
model.wv.most_similar('재미\\NNG')

[('렉\\NNP', 0.39506301283836365),
 ('만화\\NNG', 0.3715558648109436),
 ('로랑\\NNG', 0.36376771330833435),
 ('키드\\NNP', 0.3518717586994171),
 ('팬\\NNG', 0.3492623269557953),
 ('모텔\\NNG', 0.34679582715034485),
 ('사원\\NNG', 0.3385835886001587),
 ('고군분투\\NNG', 0.3375825583934784),
 ('그분\\NP', 0.3361288607120514),
 ('성질나\\VV', 0.32648980617523193)]

## 테스트 데이터셋 로드 및 pos tagging

In [14]:
test_docs = pd.read_csv('./ratings_test.txt', sep='\t')
test_docs = test_docs.dropna()

pos_docs = [pos_tagging(doc) for doc in test_docs['document']]
test_docs['prep_docs'] = pos_docs
test_docs = test_docs.dropna()
test_docs.index = range(len(test_docs))
test_docs.tail()

,id,document,label,prep_docs
49992,4608761,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1,"[오랜만\NNG, 평점\NNG, 했\VX, 킹왕짱\NNG, 쌈\NNG, 뽕\NNG,..."
49993,5308387,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO,0,"[의지\NNG, 박약\NNG, 하\VV, 거\NNB, 다\MAG, 탈영\NNG, 일..."
49994,9072549,그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다,0,"[그림\NNG, 좋\VA, 완성도\NNG, 높\VA, 보\VX, 내내\MAG, 불안..."
49995,5802125,절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네,0,"[절대\MAG, 봐서\VV, 안\MAG, 될\VV, 영화\NNG, 재미\NNG, 없..."
49996,6070594,마무리는 또 왜이래,0,"[마무리\NNG, 또\MAG, 왜\MAG, 이래\VA]"


## 테스트 결과 확인

In [30]:
def get_pred_result(doc, train_docs, topn=5, verbose=False):
    '''
    doc = 테스트 문서 (pandas DataFrame 1-row)
    train_docs = 학습 데이터셋 pandas.DataFrame (유사문서 검색을 위함.)
    topn = 학습 데이터셋에서 찾을 유사 문서의 수 (integer)
    verbose = 유사문서 및 유사도 점수 DataFrame 반환여부
    '''
    doc_content = doc[3]
    doc_label = doc[2]
    infer_vector = model.infer_vector(doc_content)
    score_threshold = int(topn/2)
    
    similar_res = [x for x in model.docvecs.most_similar([infer_vector], topn=topn)]
    similar_ids = [x[0] for x in similar_res]
    similar_scores = [x[1] for x in similar_res]
    similar_docs = train_docs[train_docs['id'].isin(similar_ids)][['document', 'label']]
    similar_docs['scores'] = similar_scores
    
    result = sum(similar_docs['label'].tolist())
    final_result = 0 if result <= score_threshold else 1
    
    if verbose:
        print('Input content :', doc[1], '\nLabel :', doc_label, '\nPrediction :', final_result)
        similar_docs.columns = ['most similar documents', 'label', 'similarity score']
        return similar_docs, final_result
    return final_result

In [31]:
# 단일문서 결과 예시
pred_result_ex, _ = get_pred_result(test_docs.iloc[50], train_docs, verbose=True)
pred_result_ex

Input content : 돈만있으면 내가 이것보단 더잘만들겠따 ㅋ 
Label : 0 
Prediction : 0


,most similar documents,label,similarity score
1838,이것두 돈들여가믄서 만들었남,0,0.767419
12509,내가 영화를 만들어도 이것보단 잘만들것다,0,0.749604
37389,나 지급이것보고있다 9876543210,1,0.734522
112673,내가만들어도 이거보다잘만들겟슴,0,0.728756
133643,내가만들어도이거보단.잘만들듯.,0,0.723171


In [32]:
# 테스트셋 sample_n개 문서에 대한 긍부정 예측 정확도 측정
count = 0
sample_n = 1000
test_array = test_docs.sample(sample_n).values

for i in range(sample_n):
    doc = test_array[i]
    label = doc[2]
    try:
        pred = get_pred_result(doc, train_docs, topn=5)
    except ValueError:
        print(doc)
    if label+pred != 1:
        count += 1
        
print('Accuracy : {}%'.format(count/sample_n * 100))

Accuracy : 77.5%


# 발생할 수 있는 문제점 및 원인

## 새로운 유형의 단어로 인해 발생하는 오류
- 원인
  - Unseen document내에 학습단계에서 정의된 말뭉치에 없는 새로운 단어는 모두 missing value
  - 중요한 단어가 학습모델 말뭉치에 없어 missing value로 처리될 경우 정확도에 부정적 영향
  - missing value로만 이루어진 문서가 입력될 경우 오류발생


- 대안예시
  - 새로운 단어에 대해 model.build_vocab('새로운 문서들', update=True)로 벡터값 부여
  <br/> (이미 존재하는 단어들의 벡터값은 업데이트 되지 않음.)

## Inference 시행 때마다 일정하지 않은 유사도 결과값 (Non-deterministic)
- 원인
  - 학습단계에서 sub-sampling이나 negative sampling이 적용될 경우 유사도 점수가 inference마다 다르게 출력
  - sub-sampling 및 negative sampling 확률적으로 단어를 학습에서 제외 또는 선택하는데, 이 무작위성이 모델링이나 inference마다 다른 결과가 나오는 원인
  
  
- 대안예시
  - sub-sampling 및 negative sampling 사용 지양 (현실적으로 어려움.)
  - epochs를 증가시키거나 여러 모델을 ensemble하여 변동 최소화

In [27]:
pred_result_ex1, _ = get_pred_result(test_docs.iloc[50], train_docs, verbose=True)
pred_result_ex1

Input content : 돈만있으면 내가 이것보단 더잘만들겠따 ㅋ 
Label : 0 
Prediction : 0


,most similar documents,label,similarity score
12509,내가 영화를 만들어도 이것보단 잘만들것다,0,0.787464
112673,내가만들어도 이거보다잘만들겟슴,0,0.775204
123218,영화 잘~ 만들었따-_-a,0,0.763742
126629,잘만들었어요.,1,0.752259
133643,내가만들어도이거보단.잘만들듯.,0,0.747282


In [19]:
pred_result_ex2, _ = get_pred_result(test_docs.iloc[50], train_docs, verbose=True)
pred_result_ex2

Input content : 돈만있으면 내가 이것보단 더잘만들겠따 ㅋ 
Label : 0 
Prediction : 0


,most similar documents,label,similarity score
11414,이것도 영화라고 만들었는지,0,0.779185
12509,내가 영화를 만들어도 이것보단 잘만들것다,0,0.769762
29786,이런걸 영화라고 만들다니,0,0.760021
95340,이것도 영화라고 만들었나?? 나참,0,0.756411
103315,.. 이런걸 영화라고 만들었나,0,0.755999
